# Semantic Parsing

We will train our own intent classifier and slot filler given a list of training questions. We will then test the model using some test questions by comparing the predicted intent and slot with the answers.

In [ ]:
parser_files = "semantic-parser"

In [ ]:
import json

train_data = []
for line in open(f'{parser_files}/train_questions_answers.txt'):
    train_data.append(json.loads(line))

# print a few examples
for i in range(5):
    print(train_data[i])
    print("-"*80)

{'question': 'Add an album to my Sylvia Plath playlist.', 'intent': 'AddToPlaylist', 'slots': {'music_item': 'album', 'playlist_owner': 'my', 'playlist': 'Sylvia Plath'}}
--------------------------------------------------------------------------------
{'question': 'add Diarios de Bicicleta to my la la playlist', 'intent': 'AddToPlaylist', 'slots': {'playlist': 'Diarios de Bicicleta', 'playlist_owner': 'my', 'entity_name': 'la la'}}
--------------------------------------------------------------------------------
{'question': 'book a table at a restaurant in Lucerne Valley that serves chicken nugget', 'intent': 'BookRestaurant', 'slots': {'restaurant_type': 'restaurant', 'city': 'Lucerne Valley', 'served_dish': 'chicken nugget'}}
--------------------------------------------------------------------------------
{'question': 'add iemand als jij to my playlist named In The Name Of Blues', 'intent': 'AddToPlaylist', 'slots': {'entity_name': 'iemand als jij', 'playlist_owner': 'my', 'playlist'

In [ ]:
test_questions = []
for line in open(f'{parser_files}/test_questions.txt'):
    test_questions.append(json.loads(line))

test_answers = []
for line in open(f'{parser_files}/test_answers.txt'):
    test_answers.append(json.loads(line))

# print a few examples
for i in range(5):
    print(test_questions[i])
    print(test_answers[i])
    print("-"*80)

Add an artist to Jukebox Boogie Rhythm & Blues
{'intent': 'AddToPlaylist', 'slots': {'music_item': 'artist', 'playlist': 'Jukebox Boogie Rhythm & Blues'}}
--------------------------------------------------------------------------------
Will it be rainy at Sunrise in Ramey Saudi Arabia?
{'intent': 'GetWeather', 'slots': {'condition_description': 'rainy', 'timeRange': 'Sunrise', 'city': 'Ramey', 'country': 'Saudi Arabia'}}
--------------------------------------------------------------------------------
Weather in two hours  in Uzbekistan
{'intent': 'GetWeather', 'slots': {'timeRange': 'in two hours', 'country': 'Uzbekistan'}}
--------------------------------------------------------------------------------
Will there be a cloud in VI in 14 minutes ?
{'intent': 'GetWeather', 'slots': {'condition_description': 'cloud', 'state': 'VI', 'timeRange': 'in 14 minutes'}}
--------------------------------------------------------------------------------
add nuba to my Metal Party playlist
{'intent': 

In [ ]:
# List of all intents
intents = set()
for example in train_data:
    intents.add(example['intent'])
print(intents)

{'BookRestaurant', 'AddToPlaylist', 'GetWeather'}


## 1: Keyword intent classifier

In this part, we will build a keyword-based intent classifier. For each intent, we extract a list of keywords that are important for that intent using the training data, and then we classify a given question into an intent using keywords. We will use the top 10 keywords. If an input question matches multiple intents, the best one is picked. If it does not match any keyword, it returns `None`.

In [ ]:
import re

# load stop words
stop_words = set()
words = open("/content/drive/My Drive/tweets/stop_words.txt", "r").read().split("\n")
for word in words:
  stop_words.add(word.strip())


def predict_intent_using_keywords(question):

  intent_dict = {} # used to store questions
  #preprocess
  for original_q in train_data:
    qu = original_q['question']
    qu = qu.lower() #lowercase the questions
    qu = re.sub("(\\d|\\W)+", " ", qu)
    qu = qu.split()
    qu = [q for q in qu if (q not in stop_words)] #remove punct and stop words

    #add question to intent dict
    if original_q['intent'] not in intent_dict:
      intent_dict[original_q['intent']] = [qu]
    else:
      intent_dict[original_q['intent']].append(qu)

  # count frequencies
  freq_list = []
  for i in intents: #initialize frequency counter
    freq = {}
    for q in intent_dict[i]:
      #print(q)
      for w in q:
        #print(w)
        if w in freq:
          freq[w] += 1
        else:
          freq[w] = 1
    freq_list.append(freq)

  #sort frequencies
  for i in range(len(freq_list)):
    freq_list[i] = dict(sorted(freq_list[i].items(), key=lambda item: item[1], reverse = True)[:10]) #take the top 10 keywords for each intent


  #preprocess question
  qu = question.lower() #lowercase the questions
  qu = re.sub("(\\d|\\W)+", " ", qu)
  qu = qu.split()
  qu = [q for q in qu if (q not in stop_words)] #remove punct and stop words

  #count the score
  score_list = []
  for f in freq_list:
    score = 0
    for q in qu:
      if q in f: #if word in question is in intent freq list
        score += f[q]
    score_list.append(score)

  if max(score_list) == 0:
    return None
  else:
    return list(intents)[score_list.index(max(score_list))] #get the intent which matches the most keywords


In [ ]:
from collections import Counter

'''Gives intent wise accuracy of your model'''
def evaluate_intent_accuracy(prediction_function_name):
  correct = Counter()
  total = Counter()
  for i in range(len(test_questions)):
    q = test_questions[i]
    gold_intent = test_answers[i]['intent']
    if prediction_function_name(q) == gold_intent:
      correct[gold_intent] += 1
    total[gold_intent] += 1
  for intent in intents:
    print(intent, correct[intent]/total[intent], total[intent])
    
# Evaluating the intent classifier. 
evaluate_intent_accuracy(predict_intent_using_keywords)

BookRestaurant 0.99 100
AddToPlaylist 1.0 100
GetWeather 0.75 100


We see that our keyword based classifier has achieved an accuracy of over 0.75 for each intent. This is fair, because there are only 3 intents and each intent's keywords are different between them. If we increase the number of keywords, we can see the accuracy getting closer to 1 too.

## 2: Statistical intent classifier

Now, let's build a statistical intent classifier. Instead of making use of keywords, we will first extract features from a given input question. In order to build a feature representation for a given sentence, we will use the word2vec embeddings of each word and take an average to represent the sentence. Then we will train a logistic regression model.

In [ ]:
import nltk
nltk.download('word2vec_sample')

[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


True

In [ ]:
from nltk.data import find
import gensim

word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np

'''Trains a logistic regression model on the entire training data. For an input question (x), the model learns to predict an intent (Y).'''
def train_logistic_regression_intent_classifier():
    lr = LogisticRegression()

    vectors = []
    intents_list = []
    #preprocess questions
    for original_q in train_data:
      intents_list.append(list(intents).index(original_q['intent'])) #get intent
      qu = original_q['question']
      qu = qu.lower() #lowercase the questions
      qu = re.sub("(\\d|\\W)+", " ", qu)
      qu = qu.split()
      qu = [q for q in qu if (q not in stop_words)] #remove punct and stop words
      
      avg_representation = [] #to store the averages of embeddings for each question

      #get word embedding for each word in question
      for w in qu:
        if w in word2vec_model:
          avg_representation.append(word2vec_model[w])
        else:
          avg_representation.append(np.zeros(word2vec_model.vector_size))

      vectors.append(np.mean(np.array(avg_representation), axis=0)) #calculate the averages

    return lr.fit(vectors, intents_list)

In [ ]:
'''For an input question, the model predicts an intent'''
def predict_intent_using_logistic_regression(question):
    lr = train_logistic_regression_intent_classifier()

    #preprocess question
    qu = question.lower() #lowercase the questions
    qu = re.sub("(\\d|\\W)+", " ", qu)
    qu = qu.split()
    qu = [q for q in qu if (q not in stop_words)] #remove punct and stop words
      
    avg_representation = [] #to store the averages of embeddings for each question

    #get word embedding for each word in question
    for w in qu:
      if w in word2vec_model:
        avg_representation.append(word2vec_model[w])
      else:
        avg_representation.append(np.zeros(word2vec_model.vector_size))
    
    return list(intents)[lr.predict(np.mean(avg_representation, axis=0).reshape(1,-1))[0]]

In [ ]:
# Evaluate the intent classifier
evaluate_intent_accuracy(predict_intent_using_logistic_regression)

AddToPlaylist 1.0 100
BookRestaurant 1.0 100
GetWeather 0.98 100


All 3 intents have accuracies close to 100%, which indicated a better performance than the keyword classifier. Word2vec takes the whole word embeddings into account to train the logistic regression, which gives us a more balanced model than just counting frequent words for each intent. 

## 3: Slot filling

We will now build a slot filling model. For now, we will only focus on the `AddToPlaylist` intent.

We are going to try to use ideas like maximum string matching to identify which slots are active and what their values are.

In [ ]:
# Let's stick to one target intent.
target_intent = "AddToPlaylist"

# This intent has the following slots
target_intent_slot_names = set()
for sample in train_data:
    if sample['intent'] == target_intent:
        for slot_name in sample['slots']:
            target_intent_slot_names.add(slot_name)
print(target_intent_slot_names)


# Extract all the relevant questions of this target intent from the test examples.
target_intent_questions = []
for i, question in enumerate(test_questions):
    if test_answers[i]['intent'] == target_intent:
        target_intent_questions.append(question)
print(len(target_intent_questions))

{'playlist_owner', 'entity_name', 'music_item', 'artist', 'playlist'}
100


In [ ]:
def initialize_slots():
    slots = {}
    for slot_name in target_intent_slot_names:
        slots[slot_name] = None
    return slots

def get_slot_values(): #get a list of values for each slot
    slot_values = {}
    for slot_name in target_intent_slot_names:
      slot_values[slot_name] = set()
      for question in train_data:
        if question['intent'] == 'AddToPlaylist':
          if slot_name in question['slots']:
            slot_values[slot_name].add(question['slots'][slot_name].lower())
    return slot_values


def predict_slot_values(question):
    slots = initialize_slots()  
    slot_values = get_slot_values() 
    for slot_name in target_intent_slot_names:
        # idenfity the slot value. By default, they are initialized to None.
        values = slot_values[slot_name]
        for v in values:
          if v in question.lower():
            slots[slot_name] = v

    return slots

def evaluate_slot_prediction_recall(slot_prediction_function):
    correct = Counter()
    total = Counter()
    # predict slots for each question
    for i, question in enumerate(target_intent_questions):
        i = test_questions.index(question) 
        gold_slots = test_answers[i]['slots']
        predicted_slots = slot_prediction_function(question)
        for name in target_intent_slot_names:
            if name in gold_slots:
                total[name] += 1.0
                if predicted_slots.get(name, None) != None and predicted_slots.get(name).lower() == gold_slots.get(name).lower(): # This line is updated after the assignment release
                    correct[name] += 1.0
    for name in target_intent_slot_names:
        print(f"{name}: {correct[name] / total[name]}")


# Our reference implementation got these numbers. You can ask others on Slack what they got.
# music_item 1.0
# playlist 0.67
# artist  0.021739130434782608
# playlist_owner 0.9444444444444444
# entity_name 0.05555555555555555

print("Slot accuracy for your slot prediction model")
evaluate_slot_prediction_recall(predict_slot_values)


Slot accuracy for your slot prediction model
playlist_owner: 0.9444444444444444
entity_name: 0.05555555555555555
music_item: 1.0
artist: 0.13043478260869565
playlist: 0.75


From the result above, it looks like maximum string matching works for some slots which can only take a limited amount of items such as the music item. However, this method does not work the best for slots like entity name and artist, because these fields are mostly names and can vary a lot. A better method to fill these slots could be to use neural networks for a better understanding of the pattern.

We will now look at some true positives, false positives, true negatives, and false negatives.

In [ ]:
# Find a true positive prediction for each slot
def true_positive(slot):
  for i, question in enumerate(target_intent_questions):
    i = test_questions.index(question)
    gold_slots = test_answers[i]['slots']
    predicted_slots = predict_slot_values(question)
    if slot in gold_slots:
      if predicted_slots.get(slot, None) != None:
        print("Question: ", end='')
        print(question)
        print("Gold answer: ", end='')
        print(gold_slots)
        print("Predicted: ", end='')
        print({slot: predicted_slots[slot]})
        print()
        break

for slot_name in target_intent_slot_names:
  true_positive(slot_name)

Question: add nuba to my Metal Party playlist
Gold answer: {'entity_name': 'nuba', 'playlist_owner': 'my', 'playlist': 'Metal Party'}
Predicted: {'playlist_owner': 'my'}

Question: Add give us rest to my 70s Smash Hits playlist.
Gold answer: {'entity_name': 'give us rest', 'playlist_owner': 'my', 'playlist': '70s Smash Hits'}
Predicted: {'entity_name': 'give us rest'}

Question: Add an artist to Jukebox Boogie Rhythm & Blues
Gold answer: {'music_item': 'artist', 'playlist': 'Jukebox Boogie Rhythm & Blues'}
Predicted: {'music_item': 'artist'}

Question: Add Roel van Velzen to my party of the century playlist.
Gold answer: {'artist': 'Roel van Velzen', 'playlist_owner': 'my', 'playlist': 'party of the century'}
Predicted: {'artist': 'roel van velzen'}

Question: Add an artist to Jukebox Boogie Rhythm & Blues
Gold answer: {'music_item': 'artist', 'playlist': 'Jukebox Boogie Rhythm & Blues'}
Predicted: {'playlist': 'blues'}



In [ ]:
# Find a false positive prediction for each slot

def false_positive(slot):
  for i, question in enumerate(target_intent_questions):
    i = test_questions.index(question)
    gold_slots = test_answers[i]['slots']
    predicted_slots = predict_slot_values(question)
    if slot not in gold_slots and predicted_slots[slot] != None:
      print("Question: ", end='')
      print(question)
      print("Gold answer: ", end='')
      print(gold_slots)
      print("Predicted: ", end='')
      print({slot: predicted_slots[slot]})
      print()
      break

for slot_name in target_intent_slot_names:
  false_positive(slot_name)

Question: add tommy johnson to The MetalSucks Playlist
Gold answer: {'artist': 'tommy johnson', 'playlist': 'The MetalSucks Playlist'}
Predicted: {'playlist_owner': 'my'}

Question: Can you put this song from Yutaka Ozaki onto my this is miles davis playlist?
Gold answer: {'music_item': 'song', 'artist': 'Yutaka Ozaki', 'playlist_owner': 'my', 'playlist': 'this is miles davis'}
Predicted: {'entity_name': 'om'}

Question: add ireland in the junior eurovision song contest 2015 to my Jazzy Dinner playlist
Gold answer: {'entity_name': 'ireland in the junior eurovision song contest 2015', 'playlist_owner': 'my', 'playlist': 'Jazzy Dinner'}
Predicted: {'music_item': 'song'}



In [ ]:
# Find a true negative prediction for each slot

def true_negative(slot):
  for i, question in enumerate(target_intent_questions):
    i = test_questions.index(question)
    gold_slots = test_answers[i]['slots']
    predicted_slots = predict_slot_values(question)
    if slot not in gold_slots:
      if predicted_slots.get(slot, None) == None:
        print("Question: ", end='')
        print(question)
        print("Gold answer: ", end='')
        print(gold_slots)
        print("Predicted: ", end='')
        print({slot: predicted_slots[slot]})
        print()
        break

for slot_name in target_intent_slot_names:
  true_negative(slot_name)

Question: Add an artist to Jukebox Boogie Rhythm & Blues
Gold answer: {'music_item': 'artist', 'playlist': 'Jukebox Boogie Rhythm & Blues'}
Predicted: {'playlist_owner': None}

Question: Add an artist to Jukebox Boogie Rhythm & Blues
Gold answer: {'music_item': 'artist', 'playlist': 'Jukebox Boogie Rhythm & Blues'}
Predicted: {'entity_name': None}

Question: add nuba to my Metal Party playlist
Gold answer: {'entity_name': 'nuba', 'playlist_owner': 'my', 'playlist': 'Metal Party'}
Predicted: {'music_item': None}

Question: Add an artist to Jukebox Boogie Rhythm & Blues
Gold answer: {'music_item': 'artist', 'playlist': 'Jukebox Boogie Rhythm & Blues'}
Predicted: {'artist': None}



In [ ]:
# Find a false negative prediction for each slot

def false_negative(slot):
  for i, question in enumerate(target_intent_questions):
    i = test_questions.index(question)
    gold_slots = test_answers[i]['slots']
    predicted_slots = predict_slot_values(question)
    if slot in gold_slots:
      if predicted_slots.get(slot, None) == None:
        print("Question: ", end='')
        print(question)
        print("Gold answer: ", end='')
        print(gold_slots)
        print("Predicted: ", end='')
        print({slot: predicted_slots[slot]})
        print()
        break

for slot_name in target_intent_slot_names:
  false_negative(slot_name)

Question: Onto jerry's Classical Moments in Movies, please add the album.
Gold answer: {'playlist_owner': "jerry's", 'playlist': 'Classical Moments in Movies', 'music_item': 'album'}
Predicted: {'playlist_owner': None}

Question: add nuba to my Metal Party playlist
Gold answer: {'entity_name': 'nuba', 'playlist_owner': 'my', 'playlist': 'Metal Party'}
Predicted: {'entity_name': None}

Question: Can you put this song from Yutaka Ozaki onto my this is miles davis playlist?
Gold answer: {'music_item': 'song', 'artist': 'Yutaka Ozaki', 'playlist_owner': 'my', 'playlist': 'this is miles davis'}
Predicted: {'artist': None}

Question: Add the album to the The Sweet Suite playlist.
Gold answer: {'music_item': 'album', 'playlist': 'The Sweet Suite'}
Predicted: {'playlist': None}

